In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv("data.csv")
X = df[['month','temp', 'RH', 'wind', 'rain']]
y = df['area']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
# import pickle


# def evaluate_model(model,curr_acc):
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)

    
#     print(f"Model: {model.__class__.__name__}")
#     accuracy =  accuracy_score(y_test, y_pred)
#     print("Accuracy:", accuracy)
#     print(classification_report(y_test, y_pred))
#     print(confusion_matrix(y_test, y_pred))
#     print("-" * 50)

#     if curr_acc < accuracy:
#         curr_acc = accuracy
#         print(f"Best Model: {model.__class__.__name__}")
#         with open("model.pkl", "wb") as file:
#             pickle.dump(model, file)
#     return curr_acc


In [6]:
import pickle


def evaluate_model(model,curr_acc):
    k = 5
    m = model
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    accuracies = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    print("Average accuracy:", accuracies.mean())
    
    if accuracies.mean() > curr_acc:
        curr_acc = accuracies.mean()
        m.fit(X, y)
        with open("model.pkl", "wb") as file:
            pickle.dump(m, file)

        print(f"Best Model: {m.__class__.__name__}")
    return curr_acc

In [7]:
best_acc = 0

In [8]:
from sklearn.linear_model import LogisticRegression
best_acc = evaluate_model(LogisticRegression(),best_acc)

Average accuracy: 0.5137821782178218
Best Model: LogisticRegression


In [9]:
from sklearn.neighbors import KNeighborsClassifier
best_acc=evaluate_model(KNeighborsClassifier(n_neighbors=5),best_acc)

Average accuracy: 0.5257425742574258
Best Model: KNeighborsClassifier


In [10]:
from sklearn.svm import SVC
best_acc=evaluate_model(SVC(kernel='linear'),best_acc)

Average accuracy: 0.5177623762376238


In [11]:
from sklearn.tree import DecisionTreeClassifier
best_acc=evaluate_model(DecisionTreeClassifier(),best_acc)

Average accuracy: 0.5416633663366337
Best Model: DecisionTreeClassifier


In [12]:
from sklearn.ensemble import RandomForestClassifier
best_acc=evaluate_model(RandomForestClassifier(n_estimators=100),best_acc)

Average accuracy: 0.5633861386138614
Best Model: RandomForestClassifier


In [13]:
from xgboost import XGBClassifier
best_acc=evaluate_model(XGBClassifier(use_label_encoder=False, eval_metric='logloss'),best_acc)

/Users/mukesh/miniconda3/envs/fp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [10:35:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/mukesh/miniconda3/envs/fp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [10:35:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/mukesh/miniconda3/envs/fp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [10:35:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/mukesh/miniconda3/envs/fp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [10:35:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  w

Average accuracy: 0.5654851485148515
Best Model: XGBClassifier


In [14]:
from sklearn.naive_bayes import GaussianNB
best_acc=evaluate_model(GaussianNB(),best_acc)

Average accuracy: 0.4741584158415842


In [15]:
from sklearn.neural_network import MLPClassifier
best_acc=evaluate_model(MLPClassifier(hidden_layer_sizes=(50,50), max_iter=500),best_acc)

Average accuracy: 0.5019207920792079


Stacking the model to see, if it performs better

In [17]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split your dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base models
base_models = [
    ('rf1', RandomForestClassifier(n_estimators=100)),
    # ('rf2', RandomForestClassifier(n_estimators=200)),
    ('dt', DecisionTreeClassifier()),
    ('xgb', XGBClassifier( eval_metric='logloss')),
    # ('KNN', KNeighborsClassifier(n_neighbors=5)),
    # ('svc', SVC(kernel='linear')),
]

# Define meta model
meta_model = RandomForestClassifier(n_estimators=100)
# meta_model = XGBClassifier( eval_metric='logloss')

# Set up the stacking classifier
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Train the model
stacking_clf.fit(X_train, y_train)

# Make predictions
y_pred = stacking_clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.5841584158415841
